# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: To gather the most relevant and up-to-date information on the latest news and updates related to inflation in the US and its impact on the stock market in 2024, I first need to search for recent articles and analyses from reputable financial news sources.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation news 2024 stock



# Agent: Lead Market Analyst
## Thought: Thought: I have now gathered detailed information from one of the most relevant articles. To further enhance my understanding and gather more information, I should read the content of another relevant article.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.usbank.com/investing/financial-perspectives/market-news/how-do-rising-interest-rates-affect-the-stock-market.html\"}"
## Tool Output: 
How Do Changing Interest Rates Affect the Stock Market? | U.S. Bank
Skip to main content
Log in
About us
Financial education
Support
Locations
Log in
 Locations
 Support
 Financial Education
 About us
Personal
 Return to Main Menu
 Personal
 Bank accounts
 Return to
 Bank accounts
 Bank accounts
Bank Smartly® Checking account
Safe Debit account
Savings accounts
Money market accounts
CDs
Debit Cards
Visa gift cards
Mobile & online features
Explore checking accounts
Explore bank accounts
 Credit cards
 Return to
 Credit car



# Agent: Lead Market Analyst
## Thought: Thought: I have now gathered detailed information from two relevant articles. To further enhance my understanding and gather more information, I should read the content of another relevant article.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.reuters.com/markets/global-markets-wrapup-1-2025-01-13/\"}"
## Tool Output: 
Stocks slip as yields rise; dollar hits highest in more than 2 years | Reuters
Skip to main contentExclusive news, data and analytics for financial market professionalsLearn more aboutRefinitivWorldBrowse WorldAfricaAmericasAsia PacificChinaEuropeIndiaIsrael and Hamas at WarJapanMiddle EastUkraine and Russia at WarUnited KingdomUnited StatesReuters NEXTBusinessBrowse BusinessAerospace & DefenseAutos & TransportationDavosEnergyEnvironmentFinanceHealthcare & PharmaceuticalsMedia & TelecomRetail & ConsumerFuture of HealthFuture of MoneyTake FiveWorld at WorkMarketsBrowse MarketsAsian MarketsCa



# Agent: Lead Market Analyst
## Final Answer: 
The current state of the financial markets, particularly in relation to inflation in the US and its impact on the stock market in 2024, is complex and influenced by various factors. 

According to recent articles, the post-Election Day rally in stock prices has proven short-lived amid forecasts that the Federal Reserve will not lower interest rates as much as expected. The Federal Reserve's decision to cut interest rates by 0.25% in December 2024, resulting in total rate cuts of 1.00% since September, has led to a mixed interest rate environment. 

This environment has had limited consequences for the broader stock market, with the S&P 500 ending the year up about 25%. However, certain equity market sectors, such as small-cap stocks, utilities, and real estate, have been negatively impacted by the rise in long-term bond yields.

The Federal Reserve's interest rate projections for 2025 are clouded by scaled-back expectations for further r



# Agent: Chief Data Strategist
## Thought: Thought: The search results provide a wealth of information on the impact of inflation on the stock market in 2024, including the latest updates on interest rates, economic indicators, and investor sentiment. However, to gain a deeper understanding of the specific sectors that have been negatively impacted by the rise in long-term bond yields, such as small-cap stocks, utilities, and real estate, I need to search for more targeted information.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"impact of inflation on small-cap stocks utilities and real estate in 2024\"}"
## Tool Output: 

Search results: Title: [PDF] Small Cap Market Review Q3 2024 - Cornerstone Investment Partners
Link: https://cornerstone-ip.com/wp-content/uploads/3Q-2024-Small-Cap-Letter-for-Website.pdf
Snippet: Yield-seeking led historically low value-creation sectors like Utilities and Real Estate to lead the market, also supporting value benchmarks

...



# Agent: Chief Data Strategist
## Thought: Thought: The search results provide valuable information on the impact of inflation on small-cap stocks, utilities, and real estate in 2024. It appears that these sectors have been negatively impacted by the rise in long-term bond yields, but there are also potential opportunities for rebound and growth. To further analyze the data and gain a deeper understanding of the market trends, I need to search for more specific information on the economic indicators and investor sentiment.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"inflation impact on stock market 2024\", \"website\": \"https://www.usbank.com\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very cl



# Agent: Chief Data Strategist
## Thought: Thought: Based on the information gathered from the previous search results, I have a good understanding of the impact of inflation on the stock market in 2024, including the effects on small-cap stocks, utilities, and real estate. However, to create a comprehensive analysis report, I need to synthesize the complex market data into actionable insights that can be transformed into compelling content. To achieve this, I will leverage the key findings from my research to identify patterns, opportunities, and risks that can inform high-impact investment strategies.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"inflation impact on stock market 2024 investment strategies\"}"
## Tool Output: 

Search results: Title: Market Update: Navigating AI Trends, Inflation Challenges, and ...
Link: https://www.bakerboyer.com/resources/articles/market-update-navigating-ai-trends-inflation-challenges-and-smart-investment-strategies
Sn



# Agent: Creative Content Director
## Thought: I need to gather current insights and data regarding US inflation and its impact on the stock market in 2024 to create high-quality content. I will perform a search to obtain the most recent articles and analyses on this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation impact on stock market 2024 analysis\"}"
## Tool Output: 

Search results: Title: Analysis: Assessing Inflation's Impact - U.S. Bank
Link: https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html
Snippet: November's inflation report showed core inflation growing by 3.3% over the previous 12 months, little changed since May 2024.
---
Title: Analysis: How will inflation, interest rates and the U.S. consumer fare ...
Link: https://www.pbs.org/newshour/economy/analysis-how-will-inflation-interest-rates-and-the-u-s-consumer-fare-in-2025
Snippet: While the recent rate cuts ma



# Agent: Creative Content Director
## Thought: Thought: I need to gather current insights and data regarding US inflation and its impact on the stock market in 2024 to create high-quality content. I will perform a search to obtain the most recent articles and analyses on this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation impact on stock market 2024 analysis\"}"
## Tool Output: 

Search results: Title: Analysis: Assessing Inflation's Impact - U.S. Bank
Link: https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html
Snippet: November's inflation report showed core inflation growing by 3.3% over the previous 12 months, little changed since May 2024.
---
Title: Analysis: How will inflation, interest rates and the U.S. consumer fare ...
Link: https://www.pbs.org/newshour/economy/analysis-how-will-inflation-interest-rates-and-the-u-s-consumer-fare-in-2025
Snippet: While the recent rat

 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfy the expect criteria, use the EXACT format below:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.




# Agent: Creative Content Director
## Final Answer: 
**Comprehensive Content on US Inflation and Its Impact on the Stock Market in 2024**

---

**Blog Post Title: Navigating 2024: The Complex Dance of US Inflation and Stock Market Dynamics**

**Introduction:**
As we move into 2024, investors find themselves navigating a landscape marked by inflationary pressures and evolving monetary policies. The Federal Reserve's actions, while seeking to stabilize the economy, have sparked significant discussions around their implications for the stock market. In th

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
📈 In 2024, the US economy is expected to grow by
2.4%, with inflation remaining above 3% in the
first half. How will this affect your investments?
#Finance #Investing #Inflation2024
--------------------------------------------------
Twitter
🔍 The Federal Reserve is positioned to cut
interest rates in late 2024, but sector
performance will vary! Stay informed about where
to invest wisely. #StockMarket #InvestmentStrategy
--------------------------------------------------
Twitter
🌍 Geopolitical conflicts and inflation may pose
risks to market stability in 2024. Are you
prepared for the impact on your portfolio?
#MarketVolatility #InvestorAwareness
--------------------------------------------------


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Navigating 2024: The Complex Dance of US Inflation and Stock Market Dynamics

## Introduction
As we move into 2024, investors find themselves navigating a landscape marked by inflationary pressures and evolving monetary policies. The Federal Reserve's actions, while seeking to stabilize the economy, have sparked significant discussions around their implications for the stock market. In this post, we will dissect how inflation impacts the stock market and what investors can expect in the coming year.

## Key Insights

### US Economic Growth and Inflation Forecast
The US economy is projected to grow by **2.4%** in 2024, despite facing challenges from persistent inflation. We anticipate the Consumer Price Index (CPI) inflation to remain above **3%** in the first half of the year before moderately decreasing to **2.7%** by year-end. The Federal Reserve is expected to cut rates twice in the latter half of 2024, signaling a gradual move toward a neutral interest rate. As inflation continues to have a foothold in the economy, understanding how it correlates with growth will be essential for investors.

Furthermore, this economic growth rate may lend a certain resilience to the stock market, balancing potential downturns from inflationary pressures. Analysts recommend monitoring the changes closely, as slight adjustments in inflation forecasts could lead to significant shifts in investor sentiment and market stability.

### Sector-Specific Impacts
Not all sectors will fare equally in 2024. Small-cap stocks, utilities, and real estate could face headwinds as rising long-term bond yields challenge their performance. However, sectors associated with technological advancement and manufacturing, bolstered by the Inflation Reduction Act, may emerge as bright spots. Investors should pay attention to market trends specific to these sectors, which could yield opportunities amidst adversity.

For instance, companies in technology and green energy are expected to benefit from government incentives and consumer shifts towards sustainable products. This divergence in sector performance necessitates a strategic approach to portfolio allocation to maximize returns while managing risk effectively.

### Geopolitical Factors and Market Stability
Geopolitical tensions, particularly in regions like Ukraine and the Middle East, together with the use of tariffs, are anticipated to keep inflation levels elevated, which could further influence the Federal Reserve's policy decisions. The interplay of these factors introduces uncertainty into market predictions, emphasizing the need for strategic planning by investors.

Investors should remain cognizant of how external conflicts impact commodity prices and overall market sentiment. Keeping abreast of these geopolitical developments can help in anticipating market fluctuations and making informed investment decisions.

## Multimedia Element: Data Visualization
*Insert an infographic showcasing the projected GDP growth, inflation rates, and interest rate predictions for 2024.*

---

## Social Media Updates

**Update 1:** 📈 In 2024, the US economy is expected to grow by 2.4%, with inflation remaining above 3% in the first half. How will this affect your investments? #Finance #Investing #Inflation2024

**Update 2:** 🔍 The Federal Reserve is positioned to cut interest rates in late 2024, but sector performance will vary! Stay informed about where to invest wisely. #StockMarket #InvestmentStrategy

**Update 3:** 🌍 Geopolitical conflicts and inflation may pose risks to market stability in 2024. Are you prepared for the impact on your portfolio? #MarketVolatility #InvestorAwareness

---

## Conclusion
The landscape for 2024 presents a mixed bag of challenges and opportunities influenced by inflation, interest rates, and geopolitical dynamics. Investors must stay vigilant and adapt their strategies accordingly. By understanding the nuances of these economic indicators and maintaining open communication with financial advisors, they can position themselves for success in the uncertain waters ahead.

## Call to Action
Stay tuned for more insights and updates as we continue to monitor the evolving financial landscape. Engage with us on our social media platforms or subscribe to our newsletter for the latest analyses and recommendations.